# Multi-GPU Mean Calculation

In [1]:
from dask.distributed import Client

import cudf, dask_cudf
from dask_cuml import mean

ImportError: cannot import name 'MGMean'

There are a couple ways to get data into cuml, which will need to be tested:
1. A large cudf object could be created and then passed to dask_cudf
2. The workers are asked to fetch the data directly

Since this will likely be running in a single worker per GPU mode, it will be important that the cuDF's are able to work across the GPUs (e.g. When a very large cuDF is partitioned across the workers- it will be important that the GPU memory is re-allocated to the new worker's local device and de-allocated on the cuDF's old device.)

__Example workflow__:
- User allocates a dask_cudf (or, eventually, a dask_cuml_array) and distributes it across the cluster
- User calls MGMean().calculate(dask_cudf) after the dask_cudf
- MGMean performs redistribution / preprocessing
- MGMean gathers allocations (hostname/device/key triplets) from Dask workers
- MGMean c++ code is executed with the allocation information as its argument


In [2]:
client = Client("10.31.241.47:8786")

In [3]:
client

Client Scheduler: tcp://10.31.241.47:8786,Cluster Workers: 8 Cores: 8 Memory: 54.10 GB


In [4]:
client.who_has()

{}

In [5]:
def create_cudf(dev):
    import numba.cuda
    import numpy as np
    numba.cuda.select_device(dev)
    logging.debug("Creating dataframe on device " + str(dev))
    return (dev, cudf.DataFrame(
        [('a', np.array([1.0, 2.0, 3.0, 4.0, 5.0, 6.0]).astype(np.float32)), 
         ('b', np.array([2.0, 3.0, 4.0, 5.0, 6.0, 7.0]).astype(np.float32))]
    ))

In [6]:
workers = list(client.has_what().keys())
workers

['tcp://10.31.241.47:45359',
 'tcp://10.31.241.47:40629',
 'tcp://10.31.241.47:39865',
 'tcp://10.31.241.47:42494',
 'tcp://10.31.241.47:34624',
 'tcp://10.31.241.47:42987',
 'tcp://10.31.241.47:43671',
 'tcp://10.31.241.47:46136']

In [7]:
import logging

logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.DEBUG)

In [8]:
# Copyright (c) 2018, NVIDIA CORPORATION.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
#
from cuml import MGMean as cumlMGMean

from tornado import gen
import dask_cudf, cudf

import logging

import time

from dask.distributed import get_worker, get_client

from dask import delayed
from collections import defaultdict
from dask.distributed import wait, default_client
import dask.dataframe as dd
import dask.array as da

import numpy as np

from toolz import first, assoc
from distributed import Client


def parse_host_port(address):
    if '://' in address:
        address = address.rsplit('://', 1)[1]
    host, port = address.split(':')
    port = int(port)
    return host, port

def to_gpu_matrix(inp):
    dev, df = inp
    select_device(dev)
    
    try:
        gpu_matrix = df.as_gpu_matrix(order='F')
        shape = df.shape[1]
        dtype = gpu_matrix.dtype
        z =np.zeros(shape, dtype=dtype)
        series = cudf.Series(z)
        gpu_array = series._column._data.to_gpu_array()
        return (dev, gpu_matrix, series, gpu_array)
    except Exception as e:
        import traceback
        logging.error("Error in to_gpu_matrix(dev=" + str(dev) + "): " + str(e))
        traceback.print_exc()
        pass

from threading import Thread
import numba.cuda
import time

def select_device(dev, close = True):
    if numba.cuda.get_current_device().id != dev:
        logging.warn("Selecting device " + str(dev))
        if close:
            numba.cuda.close()
        numba.cuda.select_device(dev)
        if dev != numba.cuda.get_current_device().id:
            logging.warn("Current device " + 
                          str(numba.cuda.get_current_device()) + 
                          " does not match expected " + str(dev))


from threading import Lock

class IPCThread(Thread):
    """
    This mechanism gets around Numba's restriction of CUDA contexts being thread-local 
    by creating a thread that can select its own device. This allows the user of IPC 
    handles to open them up directly on the same device as the owner (bypassing the 
    need for peer access.)
    """
    
    def __init__(self, ipcs, device):
        
        Thread.__init__(self)

        self.lock = Lock()
        self.ipcs = ipcs
        self.device = device
        self.running = False
        
    def run(self):
        
        select_device(self.device)
        
        print("Opening: " + str(self.device) + " " + str(numba.cuda.get_current_device()))

        self.lock.acquire()

        try:
            self.arrs = [ipc.open() for ipc in self.ipcs]
            self.ptr_info = [x.__cuda_array_interface__ for x in self.arrs]

            self.running = True
        except Exception as e:
            logging.error("Error opening ipc_handle on device " + str(self.device) + ": " + str(e))
        
        self.lock.release()

        while(self.running):
            time.sleep(0.0001)
            
        try:
            logging.warn("Closing: " + str(self.device) + str(numba.cuda.get_current_device()))
            self.lock.acquire()
            [ipc.close() for ipc in self.ipcs]
            self.lock.release()
        except Exception as e:
            logging.error("Error closing ipc_handle on device " + str(self.device) + ": " + str(e))


    def close(self):
        
        """
        This should be called before calling join(). Otherwise, IPC handles may not be 
        properly cleaned up. 
        """
        self.lock.acquire()
        self.running = False
        self.lock.release()
        
    def info(self):
        """
        Warning: this method is invoked from the calling thread. Make
        sure the context in the thread reading the memory is tied to
        self.device, otherwise an expensive peer access might take
        place underneath.
        """
        while(not self.running):
            time.sleep(0.0001)
            
        return self.ptr_info


def build_alloc_info(data):
    dev, gpu_matrix, series, gpu_array = data
    return [gpu_matrix.__cuda_array_interface__, gpu_array.__cuda_array_interface__]

def get_ipc_handles(data):
    
    dev, gpu_matrix, series, gpu_array = data

    select_device(dev)
    try:

        logging.warn("Building in_handle on " + str(dev))
        in_handle = gpu_matrix.get_ipc_handle()
        
        logging.warn("Building out_handle on " + str(dev))
        out_handle = gpu_array.get_ipc_handle()
        return (dev, in_handle, out_handle)
    except Exception as e:
        import traceback
        logging.error("Error in get_ipc_handles(dev=" + str(dev) + "): " + str(e))
        traceback.print_exc()
        pass


# Run on a single worker on each unique host
def calc_mean(data):

    ipcs, raw_arrs = data
    
    # Get device from local gpu_futures
    select_device(raw_arrs[0][0])
    
    print("begin calc_mean_device: " + str(numba.cuda.get_current_device()))

    def new_ipc_thread(dev, ipcs):
        t = IPCThread(ipcs, dev)
        t.start()
        return t
    
    open_ipcs = [new_ipc_thread(dev, [inp, outp]) for dev, inp, outp in ipcs]
    logging.debug("calc_mean_device: " + str(numba.cuda.get_current_device()))
    m = cumlMGMean()
    
    alloc_info = [t.info() for t in open_ipcs]
    alloc_info.extend([build_alloc_info(t) for t in raw_arrs])
    
    logging.debug("calc_mean_device: " + str(numba.cuda.get_current_device()))
    m.calculate(alloc_info)

    logging.debug("end calc_mean_device: " + str(numba.cuda.get_current_device()))
    return open_ipcs, raw_arrs
    
class MGMean(object):

    def calculate(self, futures):
        
        client = default_client()

        # Keep the futures around so the GPU memory doesn't get
        # deallocated on the workers.
        gpu_futures = client.sync(self._get_mg_info, futures)

        who_has = client.who_has(gpu_futures)

        key_to_host_dict = {}
        for key in who_has:
            key_to_host_dict[key] = parse_host_port(who_has[key][0])
            
        hosts_to_key_dict = {}
        for key, host in key_to_host_dict.items():
            if host not in hosts_to_key_dict:
                hosts_to_key_dict[host] = set([key])
            else:
                hosts_to_key_dict[host].add(key)

        workers = [key[0] for key in list(who_has.values())]
        hosts_dict = build_host_dict(workers)
        f = []
        for host, ports in hosts_dict.items():
            exec_node = (host, random.sample(ports, 1)[0])
            
            logging.debug("Chosen exec node is "  + str(exec_node))
            
            # Don't build an ipc_handle for exec nodes (we can just grab the local data)
            keys = set(hosts_to_key_dict[exec_node])
            
            # build ipc handles
            gpu_data_excl_worker = filter(lambda d: d[0] != exec_node, gpu_futures)
            gpu_data_incl_worker = filter(lambda d: d[0] == exec_node, gpu_futures)
            
            ipc_handles = [client.submit(get_ipc_handles, future, workers=[worker])
                           for worker, future in gpu_data_excl_worker]
            raw_arrays = [future for worker, future in gpu_data_incl_worker]
            
            logging.debug(str(ipc_handles))
            logging.debug(str(raw_arrays))
            
            f.append(client.submit(calc_mean, (ipc_handles, raw_arrays), workers = [exec_node]))

        wait(f)
        
        def close_threads(d):
            logging.debug(str("Closing threads!"))
            ipc_threads, rawarrays = d
            [t.close() for t in ipc_threads]
            
        d = [client.submit(close_threads, future) for future in f]
        wait(d)
        
        def join_threads(d):
            logging.debug(str("Joining threads!"))
            ipc_threads, rawarrays = d
            [t.join() for t in ipc_threads]
            
        d = [client.submit(join_threads, future) for future in f]
        wait(d)
        
        def print_it(data):
            dev, gpu_mat, series, gpu_arr = data
            return str(series)

        return client.gather([client.submit(print_it, future) for worker, future in gpu_futures])
    

    @gen.coroutine
    def _get_mg_info(self, futures):

        client = default_client()

        if isinstance(futures, dd.DataFrame):
            data_parts = futures.to_delayed()
            parts = list(map(delayed, data_parts))
            parts = client.compute(parts)  # Start computation in the background
            yield wait(parts)
            for part in parts:
                if part.status == 'error':
                    yield part  # trigger error locally
        else:
            data_parts = futures

        key_to_part_dict = dict([(str(part.key), part) for part in data_parts])

        who_has = yield client.who_has(data_parts)
        worker_map = []
        for key, workers in who_has.items():
            
            worker = parse_host_port(first(workers))
            worker_map.append((worker, key_to_part_dict[key]))

        gpu_data = [(worker, client.submit(to_gpu_matrix, part, workers=[worker]))
                    for worker, part in worker_map]
        
        yield wait(gpu_data)

        raise gen.Return(gpu_data)

In [9]:
from dask.distributed import wait
import random

def parse_host_port(address):
    if '://' in address:
        address = address.rsplit('://', 1)[1]
    host, port = address.split(':')
    port = int(port)
    return host, port

def build_host_dict(workers):
    hosts = set(map(lambda x: parse_host_port(x), workers))
    hosts_dict = {}
    for host, port in hosts:
        if host not in hosts_dict:
            hosts_dict[host] = set([port])
        else:
            hosts_dict[host].add(port)
            
    return hosts_dict
    

def assign_gpus():
    
    client = default_client()
    
    """
    Supports a multi-GPU & multi-Node environment by assigning a single local GPU
    to each worker in the cluster. This is necessary due to Numba's restriction
    that only a single CUDA context (and thus a single device) can be active on a 
    thread at a time. 
    
    The GPU assignments are valid as long as the future returned from this function
    is held in scope. This allows any functions that need to allocate GPU data to
    utilize the CUDA context on the same device, otherwise data could be lost.
    """

    workers = list(client.has_what().keys())
    hosts_dict = build_host_dict(workers)
    
    print(str(hosts_dict))
    
    def get_gpu_info():
        import numba.cuda
        return [x.id for x in numba.cuda.gpus]
    
    gpu_info = dict([(host, client.submit(get_gpu_info, 
                     workers = [(host, random.sample(hosts_dict[host], 1)[0])])) 
                     for host in hosts_dict])
    wait(list(gpu_info.values()))
    
    # Scatter out a GPU device ID to workers
    f = []
    for host, future in gpu_info.items():
        gpu_ids = future.result()
        ports = random.sample(hosts_dict[host], min(len(gpu_ids), len(hosts_dict[host])))
        
        f.extend([client.scatter(device_id, workers = [(host,port)]) for device_id, port in zip(gpu_ids, ports)])
        
    wait(f)
        
    return f

In [10]:
assignments = assign_gpus()

{'10.31.241.47': {34624, 42987, 45359, 40629, 43671, 46136, 39865, 42494}}


In [11]:
client.who_has()

{'int-06e5a71c9839bd98760be56f629b24cc': ('tcp://10.31.241.47:42494',),
 'int-58e78e1b34eb49a68c65b54815d1b158': ('tcp://10.31.241.47:34624',),
 'int-5c8a950061aa331153f4a172bbcbfd1b': ('tcp://10.31.241.47:46136',),
 'int-5cd9541ea58b401f115b751e79eabbff': ('tcp://10.31.241.47:43671',),
 'int-7ec5d3339274cee5cb507a4e4d28e791': ('tcp://10.31.241.47:40629',),
 'int-c0a8a20f903a4915b94db8de3ea63195': ('tcp://10.31.241.47:39865',),
 'int-ce9a05dd6ec76c6a6d171b0c055f3127': ('tcp://10.31.241.47:42987',),
 'int-d3395e15f605bc35ab1bac6341a285e2': ('tcp://10.31.241.47:45359',)}

In [12]:
res = [client.submit(create_cudf, future, workers = [worker]) for future, worker in zip(assignments, workers)]
wait(res)

DoneAndNotDoneFutures(done={<Future: status: finished, type: tuple, key: create_cudf-d54089d155c6bb74354100a1920225a4>, <Future: status: finished, type: tuple, key: create_cudf-ff3f8953e808d4bdf5c6b5a626ad9ced>, <Future: status: finished, type: tuple, key: create_cudf-9f468e754fec35f10ff68e895241e6e7>, <Future: status: finished, type: tuple, key: create_cudf-ff053e7be3d0887b01fc04c312563d9c>, <Future: status: finished, type: tuple, key: create_cudf-05cc98e6e07988f1b0844cbc69e051d4>, <Future: status: finished, type: tuple, key: create_cudf-0865da5f3751794ff89a9cfd76ddab89>, <Future: status: error, key: create_cudf-47da4823f4da337847849aab9e42b2a9>, <Future: status: finished, type: tuple, key: create_cudf-cfdcd1dc4687751839df9dfba915bb21>}, not_done=set())

In [13]:
m = MGMean()

In [14]:
m.calculate(res)

RuntimeError: generator raised StopIteration

In [14]:
client.who_has()

{'create_cudf-01cfabc552f4f767c004c9b8dc0b092d': ('tcp://10.31.241.47:45359',),
 'create_cudf-5b6ddb98e5e8fa9d8804bb716780e75e': ('tcp://10.31.241.47:40629',),
 'create_cudf-97838ce0467ab5343135d1d9c4d38b51': ('tcp://10.31.241.47:42494',),
 'create_cudf-a6be9ce53f0a0f356ed90cb9f989b1ee': ('tcp://10.31.241.47:34624',),
 'create_cudf-bf0133384caf5a1fe2ddc94a81de3747': ('tcp://10.31.241.47:46136',),
 'create_cudf-e525748e1f6b3bf2fe3ce65e58eb7745': ('tcp://10.31.241.47:43671',),
 'create_cudf-f7df963bd0c86160c4fc45cb71bb62b3': ('tcp://10.31.241.47:39865',),
 'create_cudf-fbd46cf372eecceab73318b3f5689759': ('tcp://10.31.241.47:42987',),
 'int-06e5a71c9839bd98760be56f629b24cc': ('tcp://10.31.241.47:34624',
  'tcp://10.31.241.47:40629'),
 'int-58e78e1b34eb49a68c65b54815d1b158': ('tcp://10.31.241.47:39865',),
 'int-5c8a950061aa331153f4a172bbcbfd1b': ('tcp://10.31.241.47:45359',
  'tcp://10.31.241.47:46136'),
 'int-5cd9541ea58b401f115b751e79eabbff': ('tcp://10.31.241.47:45359',
  'tcp://10.31.2

In [23]:
print(str(result))

NameError: name 'result' is not defined

In [174]:
client.who_has()



{'create_cudf-0e48e7373118138593b3b44a113a85e4': ('tcp://10.31.241.47:33707',),
 'create_cudf-1006dc4a9f499abd1299a8d5366e9419': ('tcp://10.31.241.47:37111',),
 'create_cudf-1bce8d98ea489bc535ca6b583b152c9a': ('tcp://10.31.241.47:40989',),
 'create_cudf-21d407fb894df1cef687d30ca9e60d10': ('tcp://10.31.241.47:33489',),
 'create_cudf-4356fccf4b64c722f137260005ecc8d3': ('tcp://10.31.241.47:37111',),
 'create_cudf-4574330d22f51c1a659f681a578d912c': ('tcp://10.31.241.47:37817',),
 'create_cudf-558ba73530133f681c17863e2ce3701b': ('tcp://10.31.241.47:44554',),
 'create_cudf-5ca6b344498c49aab2fe715a98e9c550': ('tcp://10.31.241.47:33489',),
 'create_cudf-724f7b73a573c1763c187def2147ec4e': ('tcp://10.31.241.47:33707',),
 'create_cudf-7bc558daec6bf3543ca793024d89c051': ('tcp://10.31.241.47:37817',),
 'create_cudf-8082bdccc0dcd5caf09e25f03dd71037': ('tcp://10.31.241.47:35132',),
 'create_cudf-9c17a4f904642d9c2c5a28b1518f286f': ('tcp://10.31.241.47:35132',),
 'create_cudf-b0f39a9aafb70801d15fe47423